In [1]:
import sys 
import time
import cv2
import os
import numpy as np
import onnxruntime as ort
import matplotlib.pyplot as plt
# sys.path.append(r"C:\Users\aup\Desktop\AIPC_Toturials\utils")
from utils.yolov8_utils import get_labels_from_txt , v8_draw , v8_nms


# 获取当前可用的 Execution Providers
available_providers = ort.get_available_providers()

print("Available Execution Providers:")
print(available_providers)

os.environ["HSA_OVERRIDE_GFX_VERSION"] = "11.0.0"

Available Execution Providers:
['MIGraphXExecutionProvider', 'ROCMExecutionProvider', 'CPUExecutionProvider']


In [2]:

image_path = r"temp/person.jpg"  # 替换为您的测试图像路径
model = r"toturials_models/yolov8/DetectionModel_int.onnx"
session = ort.InferenceSession(model,providers=["ROCMExecutionProvider"])

2025-01-13 10:34:20.773031387 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-01-13 10:34:20.773075517 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [3]:
height, width = 640, 640
img0 = cv2.imread(image_path)
x_scale = img0.shape[1] / width
y_scale = img0.shape[0] / height
img = img0[:, :, ::-1]
img = img / 255.
img = cv2.resize(img, (width, height))
img = np.transpose(img, (2, 0, 1))
data = np.expand_dims(img, axis=0)


In [4]:


input_name = session.get_inputs()[0].name
label_name = session.get_outputs()[0].name
pred = session.run([label_name], {input_name: data.astype(np.float32)})[0]

In [ ]:
pred = np.squeeze(pred)
pred = np.transpose(pred, (1, 0))
pred_class = pred[..., 4:]
pred_conf = np.max(pred_class, axis=-1)
pred = np.insert(pred, 4, pred_conf, axis=-1)
result = v8_nms(pred, 0.3, 0.45)
labels_dict = get_labels_from_txt('toturials_models/yolov8/coco_names.txt')  # 得到类别信息，返回序号与类别对应的字典
ret_img = v8_draw(img0, x_scale, y_scale, result,labels_dict)
cv2.imwrite("temp/v8_output_file.jpg", ret_img)

In [9]:

process_output_and_save(output, img, conf_threshold=0.25, iou_threshold=0.45, save_path="result.jpg")